# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f7d880e8be0>
├── 'a' --> tensor([[ 0.5397,  0.8544, -0.8882],
│                   [ 0.8508, -0.5312, -1.1660]])
└── 'x' --> <FastTreeValue 0x7f7d880e8490>
    └── 'c' --> tensor([[-1.1085, -1.0771, -0.7692, -0.6660],
                        [ 0.3262,  1.8130,  1.8673,  1.4062],
                        [-0.6533, -0.2896,  0.0958, -0.4527]])

In [4]:
t.a

tensor([[ 0.5397,  0.8544, -0.8882],
        [ 0.8508, -0.5312, -1.1660]])

In [5]:
%timeit t.a

58 ns ± 0.107 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f7d880e8be0>
├── 'a' --> tensor([[ 1.1919,  1.5332, -2.3650],
│                   [-1.2802,  0.4827, -0.4414]])
└── 'x' --> <FastTreeValue 0x7f7d880e8490>
    └── 'c' --> tensor([[-1.1085, -1.0771, -0.7692, -0.6660],
                        [ 0.3262,  1.8130,  1.8673,  1.4062],
                        [-0.6533, -0.2896,  0.0958, -0.4527]])

In [7]:
%timeit t.a = new_value

64.9 ns ± 0.0317 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.5397,  0.8544, -0.8882],
               [ 0.8508, -0.5312, -1.1660]]),
    x: Batch(
           c: tensor([[-1.1085, -1.0771, -0.7692, -0.6660],
                      [ 0.3262,  1.8130,  1.8673,  1.4062],
                      [-0.6533, -0.2896,  0.0958, -0.4527]]),
       ),
)

In [10]:
b.a

tensor([[ 0.5397,  0.8544, -0.8882],
        [ 0.8508, -0.5312, -1.1660]])

In [11]:
%timeit b.a

59.2 ns ± 0.0269 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.0125,  1.6864, -0.7203],
               [-1.1089,  0.7984, -0.7748]]),
    x: Batch(
           c: tensor([[-1.1085, -1.0771, -0.7692, -0.6660],
                      [ 0.3262,  1.8130,  1.8673,  1.4062],
                      [-0.6533, -0.2896,  0.0958, -0.4527]]),
       ),
)

In [13]:
%timeit b.a = new_value

484 ns ± 0.417 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

839 ns ± 0.828 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 12.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

136 µs ± 392 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

138 µs ± 547 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f7cd3371d00>
├── 'a' --> tensor([[[ 0.5397,  0.8544, -0.8882],
│                    [ 0.8508, -0.5312, -1.1660]],
│           
│                   [[ 0.5397,  0.8544, -0.8882],
│                    [ 0.8508, -0.5312, -1.1660]],
│           
│                   [[ 0.5397,  0.8544, -0.8882],
│                    [ 0.8508, -0.5312, -1.1660]],
│           
│                   [[ 0.5397,  0.8544, -0.8882],
│                    [ 0.8508, -0.5312, -1.1660]],
│           
│                   [[ 0.5397,  0.8544, -0.8882],
│                    [ 0.8508, -0.5312, -1.1660]],
│           
│                   [[ 0.5397,  0.8544, -0.8882],
│                    [ 0.8508, -0.5312, -1.1660]],
│           
│                   [[ 0.5397,  0.8544, -0.8882],
│                    [ 0.8508, -0.5312, -1.1660]],
│           
│                   [[ 0.5397,  0.8544, -0.8882],
│                    [ 0.8508, -0.5312, -1.1660]]])
└── 'x' --> <FastTreeValue 0x7f7cd3371f40>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.3 µs ± 51.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f7cd33bfcd0>
├── 'a' --> tensor([[ 0.5397,  0.8544, -0.8882],
│                   [ 0.8508, -0.5312, -1.1660],
│                   [ 0.5397,  0.8544, -0.8882],
│                   [ 0.8508, -0.5312, -1.1660],
│                   [ 0.5397,  0.8544, -0.8882],
│                   [ 0.8508, -0.5312, -1.1660],
│                   [ 0.5397,  0.8544, -0.8882],
│                   [ 0.8508, -0.5312, -1.1660],
│                   [ 0.5397,  0.8544, -0.8882],
│                   [ 0.8508, -0.5312, -1.1660],
│                   [ 0.5397,  0.8544, -0.8882],
│                   [ 0.8508, -0.5312, -1.1660],
│                   [ 0.5397,  0.8544, -0.8882],
│                   [ 0.8508, -0.5312, -1.1660],
│                   [ 0.5397,  0.8544, -0.8882],
│                   [ 0.8508, -0.5312, -1.1660]])
└── 'x' --> <FastTreeValue 0x7f7cd3368f40>
    └── 'c' --> tensor([[-1.1085, -1.0771, -0.7692, -0.6660],
                        [ 0.3262,  1.8130,  1.8673,  1.4062],
                 

In [23]:
%timeit t_cat(trees)

30.1 µs ± 72.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.7 µs ± 77.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.5397,  0.8544, -0.8882],
                [ 0.8508, -0.5312, -1.1660]],
       
               [[ 0.5397,  0.8544, -0.8882],
                [ 0.8508, -0.5312, -1.1660]],
       
               [[ 0.5397,  0.8544, -0.8882],
                [ 0.8508, -0.5312, -1.1660]],
       
               [[ 0.5397,  0.8544, -0.8882],
                [ 0.8508, -0.5312, -1.1660]],
       
               [[ 0.5397,  0.8544, -0.8882],
                [ 0.8508, -0.5312, -1.1660]],
       
               [[ 0.5397,  0.8544, -0.8882],
                [ 0.8508, -0.5312, -1.1660]],
       
               [[ 0.5397,  0.8544, -0.8882],
                [ 0.8508, -0.5312, -1.1660]],
       
               [[ 0.5397,  0.8544, -0.8882],
                [ 0.8508, -0.5312, -1.1660]]]),
    x: Batch(
           c: tensor([[[-1.1085, -1.0771, -0.7692, -0.6660],
                       [ 0.3262,  1.8130,  1.8673,  1.4062],
                       [-0.6533, -0.2896,  0.0958, -0.4527]],
         

In [26]:
%timeit Batch.stack(batches)

78.4 µs ± 219 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.5397,  0.8544, -0.8882],
               [ 0.8508, -0.5312, -1.1660],
               [ 0.5397,  0.8544, -0.8882],
               [ 0.8508, -0.5312, -1.1660],
               [ 0.5397,  0.8544, -0.8882],
               [ 0.8508, -0.5312, -1.1660],
               [ 0.5397,  0.8544, -0.8882],
               [ 0.8508, -0.5312, -1.1660],
               [ 0.5397,  0.8544, -0.8882],
               [ 0.8508, -0.5312, -1.1660],
               [ 0.5397,  0.8544, -0.8882],
               [ 0.8508, -0.5312, -1.1660],
               [ 0.5397,  0.8544, -0.8882],
               [ 0.8508, -0.5312, -1.1660],
               [ 0.5397,  0.8544, -0.8882],
               [ 0.8508, -0.5312, -1.1660]]),
    x: Batch(
           c: tensor([[-1.1085, -1.0771, -0.7692, -0.6660],
                      [ 0.3262,  1.8130,  1.8673,  1.4062],
                      [-0.6533, -0.2896,  0.0958, -0.4527],
                      [-1.1085, -1.0771, -0.7692, -0.6660],
                      [ 0.3262,  

In [28]:
%timeit Batch.cat(batches)

142 µs ± 457 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

308 µs ± 1.59 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
